In [1]:
import pandas as pd
import re
import json
from pandas.io.json import json_normalize
import requests
from bs4 import BeautifulSoup

In [325]:
###Scrape a complete part list###

#Get content and find all a[href]
def getHref(url):
    soupOverview = BeautifulSoup(requests.get(url).text, 'html.parser')
    urls = soupOverview.select('#content_block')[0].findAll('a', href=True)
    return urls

#Settings
tisUrlOverview = ""
filename = tisUrlOverview.split("parts-catalog/")[1].split("/browse")[0].replace('/', "_").replace('.', "").replace('-', "_")
baseUrl = ""

#Get all 1st level category URLs and parse them to object
categories = []
for e in getHref(tisUrlOverview):
    if "browse" in e['href']: 
        categories.append({
            "title" : e.text.strip(),
            "url" : baseUrl+e['href'],
            "subcategories" : [], 
        })
print(len(categories), "categories fetched!")

#Get all subcategories URLs
scCounter = 0
for i in range(0, len(categories)):
    for e in getHref(categories[i].get('url')):
        if "browse" in e['href']: 
            categories[i]["subcategories"].append({
                "title" : e.text.strip(),
                "url" : baseUrl+e['href'], 
                "parts" : [],     
            })
            scCounter += 1
print(str(scCounter), "subcategories fetched!")

#For all Categories
parts = []
part = {}
partVariation = ""
for i in range(0, len(categories)):
    for z in range(0, len(categories[i].get('subcategories'))-1):
        url = categories[i].get('subcategories')[z].get('url')
        partsTable = BeautifulSoup(requests.get(url).text, 'html.parser').select('#parts_table')[0].find('tbody')
    
        #Parse parts page
        part = {}
        partVariation = ""
        for e in partsTable.findAll('tr', recursive=False)[0:]:

            #Get name of the part from "title_row" 
            if e.select('.title_row') and len(e.select('.small_text')[0].text) > 0:
                partName = e.select('.small_text')[0].text
                if len(e.select('.small_text')) > 1:
                    partVariation = e.select('.small_text')[1].text 

            #Get other Columns
            if (not e.select('.title_row') and len(e.text.strip())>0 and e.get("id") != "accent_tr"):
                part = {}
                part["name"] = partName 
                part["id"] = e.select('.clipboard_part')[0].get('value')
                part["note"] = e.select('.clipboard_part')[0].findNext('td', {"class": "part_td"}).text.strip().replace('\n', ' ')
                if len(e.select(".add_td"))> 0 and 'Für Fahrzeuge mit' in e.select(".add_td")[0].text:
                    partVariation = e.select(".add_td")[0].select("div.mobile.only")[0].select("div.drop_info.hide")[0].text

                if len(partVariation) > 0 and 'Für Fahrzeuge mit' in partVariation:
                    part["variation"] = re.split('Für Fahrzeuge mit', partVariation)[1].replace('\n','').replace('', '').replace("Ja", "Ja ").replace("Nein", "Nein ")
                else:
                    part["variation"] = ""
                part["amount"] = e.select("td.light.part_td.small.center_text")[0].text.strip().replace('\n', ' ')    
                part["category"] = categories[i].get('title')
                part["categoryURL"] = categories[i].get('url')
                part["subcategory"] = categories[i].get('subcategories')[z].get('title')
                part["subcategoryURL"] = categories[i].get('subcategories')[z].get('url')
                parts.append(part)

            #Reset new Part
            if (e.get("id") == "accent_tr"):
                partVariation = ""
                partName = ""
        print(categories[i].get('subcategories')[z].get('title'), "succesfull crawled")
    print(categories[i].get('title'), "succesfull crawled")
    
#Export to DataFrame
data = json.dumps(parts)
json_data = json.loads(data)
df = json_normalize(json_data)
df.to_csv(filename + '.csv', index = false)

36 categories fetched!
659 subcategories fetched!
Betriebsanleitung E85, E86 succesfull crawled
Technikinformationen BMW succesfull crawled
Pflegehinweise BMW succesfull crawled
Radio E83, E85, E86 succesfull crawled
Telefon Festeinbau, SA630/631/638 succesfull crawled
Universal Handyvorbereitung BMW succesfull crawled
Navigation und TV, SA609 + SA601 succesfull crawled
Navigationssystem Business E83, E85, E86 succesfull crawled
EG-Übereinstimmungsbescheinigung succesfull crawled
Bordbuchetui BMW succesfull crawled
Bordbuchetui BMW mit Prägung succesfull crawled
Zusatzbetriebsanleitung Australien succesfull crawled
Serviceheft 2004 — 2008 BMW succesfull crawled
Batterieladekalender succesfull crawled
Montageinfos, Bedienungsanleitungen BMW succesfull crawled
Montagehinweise BMW succesfull crawled
Schablonen BMW succesfull crawled
Formular BMW succesfull crawled
Anhänger BMW succesfull crawled
Aufkleber BMW succesfull crawled
Etiketten BMW succesfull crawled
CD/DVD Sonstiges BMW succesf

Schwungrad Automatik succesfull crawled
Steuertrieb-Steuerkette unten succesfull crawled
Steuertrieb-Steuerkette oben/Einlass succesfull crawled
Steuertrieb-Steuerkette oben/Auslass succesfull crawled
Ventilsteuerung-Nockenwelle/Schmierung succesfull crawled
Ölversorgung/Ölpumpe mit Antrieb succesfull crawled
Ölversorgung-Ölfilter succesfull crawled
Kühlsystem-Wasserpumpe/Thermostat succesfull crawled
Kühlsystem-Wasserschlauchführung succesfull crawled
Sauganlage succesfull crawled
Sauganlage-Anbauteile succesfull crawled
Unterdrucksteuerung-Motor succesfull crawled
Abgasschadstoff-Reduzierung-Luftpumpe succesfull crawled
Motor succesfull crawled
Zündspule/Zündkerze succesfull crawled
Sensoren succesfull crawled
Steuergerät programmiert DME succesfull crawled
Grundsteuergerät DME / MS45.0 succesfull crawled
Kompakt Generator succesfull crawled
Generator Einzelteile 120A Bosch succesfull crawled
Anlasser succesfull crawled
Kabelbaum Motor Getriebemodul succesfull crawled
Kabelbaum Motor

Gepäckraumboden/Radhaus hinten succesfull crawled
Bodenteile hinten succesfull crawled
Seitenwand/Heckverkleidung succesfull crawled
Frontklappe/Anbauteile succesfull crawled
Tür vorn — Scharnier/Türbremse succesfull crawled
Schliesssystem Tür vorn succesfull crawled
Türdichtung vorne succesfull crawled
Gleichschliessung succesfull crawled
Schlüssel mit Akku succesfull crawled
Heckklappe/Einzelteile succesfull crawled
Mechanische Verbindungselemente succesfull crawled
Reparaturkit Gewindereparatur succesfull crawled
Reparaturkit Gewindereparatur Dünnwand succesfull crawled
Karosserie Blechteile succesfull crawled
Verglasung succesfull crawled
Aussenspiegel succesfull crawled
Aussenspiegel (S760A) succesfull crawled
Spiegelglas succesfull crawled
Türfenstermechanismus vorn succesfull crawled
Träger vorn succesfull crawled
Verkleidung vorn succesfull crawled
Blende Schweller / Radlauf succesfull crawled
Träger hinten succesfull crawled
Verkleidung hinten succesfull crawled
Äussere Blende

Stellantrieb Heizung succesfull crawled
Verdampfer / Expansionsventil succesfull crawled
Gehäuseteile Klimaautomatik succesfull crawled
Gehäuseteile Heizung succesfull crawled
Gehäuseteile Klimaanlage succesfull crawled
Satz Kleinteile Heiz/Klimaanlagegehäuse succesfull crawled
Satz Kleinteile Klimaautomatikgehäuse succesfull crawled
Gebläse / Anbauteile succesfull crawled
Kabelsatz Heizgerät/Klimaanlage succesfull crawled
Temperatursensor succesfull crawled
Stellantrieb Klimaautomatik succesfull crawled
Stellantrieb Klimaanlage succesfull crawled
Stellantrieb Heizung succesfull crawled
Sensor für AUC succesfull crawled
Klimakompressor succesfull crawled
Lagerbock Klimakompressor succesfull crawled
Kondensator Klimaanlage succesfull crawled
Kältemittelleitungen succesfull crawled
Trocknerflasche succesfull crawled
Diverse Kabeltüllen succesfull crawled
Anbauteile Heiz/Klimagerät succesfull crawled
Heizung und Klimaanlage succesfull crawled
Radio BMW Basic CD succesfull crawled
Radio BM

In [391]:
#Setup
cat_compare = "Bremsen"
#subcat_compare = "Hinterradbremse-Bremsscheibe"

#Chose Cars and Read Files
df_c_initial = pd.read_csv('E85_25_M54.csv')
df_c_compare = pd.read_csv('E85_30_M54.csv')

#Chose Category to Compare and Trim DF
df_c_initial = df_c_initial[df_c_initial.category == cat_compare]
df_c_compare = df_c_compare[df_c_compare.category == cat_compare]

#Chose Subcategory to Compare and Trim DF
if subcat_compare:
    df_c_initial = df_c_initial[df_c_initial.subcategory == subcat_compare]
    df_c_compare = df_c_compare[df_c_compare.subcategory == subcat_compare]

#Compare Both DF
df_compared = pd.DataFrame()
for tn in df_c_compare.id:
    if len(df_c_initial[df_c_initial.id == tn]) == 0:
        df_compared = df_compared.append(df_c_compare[df_c_compare.id == tn])
df_compared

,name,id,note,variation,amount,category,categoryURL,subcategory,subcategoryURL
2472,Bremsträger,34106758120,NaN,NaN,2,Bremsen,http://de.bmwfans.info//parts-catalog/E85/Euro...,Vorderradbremse-Bremsbelag-Fühler,http://de.bmwfans.info//parts-catalog/E85/Euro...
2486,Bremsscheibe belüftet,34116794302,300X22,NaN,2,Bremsen,http://de.bmwfans.info//parts-catalog/E85/Euro...,Vorderradbremse-Bremsscheibe,http://de.bmwfans.info//parts-catalog/E85/Euro...
2487,Bremsscheibe belüftet,34116864058,300X22,NaN,2,Bremsen,http://de.bmwfans.info//parts-catalog/E85/Euro...,Vorderradbremse-Bremsscheibe,http://de.bmwfans.info//parts-catalog/E85/Euro...
2492,Bremssattelgehäuse links,34216758135,NaN,NaN,1,Bremsen,http://de.bmwfans.info//parts-catalog/E85/Euro...,Hinterradbremse-Bremsbelag-Fühler,http://de.bmwfans.info//parts-catalog/E85/Euro...
2493,Bremssattelgehäuse rechts,34216758136,NaN,NaN,1,Bremsen,http://de.bmwfans.info//parts-catalog/E85/Euro...,Hinterradbremse-Bremsbelag-Fühler,http://de.bmwfans.info//parts-catalog/E85/Euro...
2494,Bremsträger,34216758134,NaN,NaN,2,Bremsen,http://de.bmwfans.info//parts-catalog/E85/Euro...,Hinterradbremse-Bremsbelag-Fühler,http://de.bmwfans.info//parts-catalog/E85/Euro...
2504,Reparatursatz Bremsbeläge asbestfrei,34216778168,NaN,NaN,1,Bremsen,http://de.bmwfans.info//parts-catalog/E85/Euro...,Hinterradbremse-Bremsbelag-Fühler,http://de.bmwfans.info//parts-catalog/E85/Euro...
2509,Bremsscheibe belüftet,34216794303,294X19,NaN,2,Bremsen,http://de.bmwfans.info//parts-catalog/E85/Euro...,Hinterradbremse-Bremsscheibe,http://de.bmwfans.info//parts-catalog/E85/Euro...
